ISMB 2024 Tutorial: Multi-omic data integration for microbiome research using scikit-bio

# Section 01: Basic bioinformatics using scikit-bio

- Time: 9:00 - 10:00 EDT, Jul 11, 2024
- Instructor: Matthew Aton

---

Published by scikit-bio [here](https://github.com/scikit-bio/scikit-bio-tutorials/blob/247a4a25f1568ab088d8a152a63074dabb3cf092/01-basic-bioinfo/01-basic-bioinfo.ipynb).

Welcome to the tutorial for using the Python package **scikit-bio** (https://scikit.bio) for omic data analysis and microbiome research! This section serves as an introduction to scikit-bio. It will guide you through the installation, and give you the first glimpse of scikit-bio's flavor and functionality by walking through a very basic series of bioinformatic analyses: sequence, alignment, and phylogeny.

## Preparation

*Editor's note: I removed the setup section from the original document so as to avoid confusion. Please see the original [here](https://github.com/scikit-bio/scikit-bio-tutorials/blob/247a4a25f1568ab088d8a152a63074dabb3cf092/01-basic-bioinfo/01-basic-bioinfo.ipynb) if you would like to see their installation methods.*

Check if scikit-bio can be imported into Python.

In [ ]:
import skbio
skbio.__version__

### Access data files

Data files to be used during the tutorial may be accessed via any of the three methods, depending on your preference.

#### 1. Download as needed during the tutorial

In [ ]:
## These do not need to be run, the data should already be in the 'data' folder.
## If it isn't, then run this code and move the 'intro' folder into the 'data' folder.
# !wget -q -O ../data/intro.tar.gz "https://www.dropbox.com/scl/fi/1oou3hrzebog6jhb06nqq/intro.tar.gz?rlkey=8lt3oabfkdp738xy4wy5cdg1t&dl=0"
# !tar zxf ../data/intro.tar.gz

In [ ]:
HOME = '../data/intro'

In [ ]:
!ls $HOME

Let's mute some warnings.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Reading sequences

The downloaded data package has a single file: `il6.ffn`. It is a [FASTA](https://en.wikipedia.org/wiki/FASTA_format) file containing the coding sequences (CDSes) of the [Interleukin 6](https://en.wikipedia.org/wiki/Interleukin_6) (**IL6**) gene of seven representative organisms.

- _Homo sapiens_ (human)
- _Pan troglodytes_ (chimp)
- _Macaca mulatta_ (monkey)
- _Sus scrofa_ (pig)
- _Mus musculus_ (mouse)
- _Rattus norvegicus_ (rat)
- _Gallus gallus_ (chicken)

We will read all sequences from the file into Python. These are DNA sequences. scikit-bio's [`DNA`](https://scikit.bio/docs/latest/generated/skbio.sequence.DNA.html) class can handle them.

In [ ]:
from skbio import DNA

Since we ultimately want a list of `DNA` sequences, we instantiate an empty list and append each sequence to that list using a for loop and `skbio.io.read`. By setting the `constructor` parameter to `DNA`, read will automatically convert each sequence into a scikit-bio `DNA` object. For protein or RNA sequences, the constructor can be set to `Protein` or `RNA`, respectively.

In [ ]:
seqs = []
for seq in skbio.io.read(f'{HOME}/il6.ffn', format='fasta', constructor=DNA):
    seqs.append(seq)

## Basic sequence types

scikit-bio supports the three basic sequence types of biology: DNA, RNA, and Protein. Additionally, each type of sequence may include optional metadata and positional metadata, which are each mutable.

We now have a list of sequences. Python list indexing can be used to show us the first two sequences in our list. They are the human and mouse IL6 coding regions. Let's explore each of these a bit more to see what functionality scikit-bio offers.

In [ ]:
human = seqs[0]
human

In [ ]:
mouse = seqs[1]
mouse

The primary information stored for each type of sequence object is the underlying sequence data itself, which is stored as an immutable NumPy array of 8-bit integers. This uses ASCII encoding and offers significant performance gains compared to operating on the strings of letters which compose the sequences.

In [ ]:
human._bytes[:100]

Common operations are defined as methods, for example computing the **reverse complement** of a DNA sequence, or searching for N-glycosylation motifs in protein sequences. Class attributes provide valid character sets, complement maps for different sequence types, and degenerate character definitions.

In [ ]:
human.reverse_complement()

We can also **transcribe** and **translate** sequences with built-in methods of the sequence objects.

In [ ]:
human.transcribe()

In [ ]:
human.translate()

## Pairwise Alignment

scikit-bio provides methods for [sequence alignment](https://en.wikipedia.org/wiki/Sequence_alignment). We will start by showing the functionality of **local pairwise alignment** using the Striped Smith-Waterman (SSW) algorithm, as implemented in [`local_pairwise_align_ssw`](https://scikit.bio/docs/latest/generated/skbio.alignment.local_pairwise_align_ssw.html). The output of the function provides the alignment, the alignment score, and the start and end positions for the two sequences in the alignment.

- Upgrade of alignment algorithms in scikit-bio is currently in progress to make them more efficient and usable.

In [ ]:
from skbio.alignment import local_pairwise_align_ssw

In [ ]:
align, score, pos = local_pairwise_align_ssw(human, mouse)

In [ ]:
score

In [ ]:
pos

The alignment is a tabular multiple sequence alignment ([`TabularMSA`](https://scikit.bio/docs/latest/generated/skbio.alignment.TabularMSA.html)) object. It is a data structure meant for intuitive manipulation and visualization of pairwise or multiple sequence alignments.

In [ ]:
align

### Alignment path

scikit-bio further provides [`AlignPath`](https://scikit.bio/docs/latest/generated/skbio.alignment.AlignPath.html) (and its derivative `PairAlignPath`), a more memory and compute efficient data structure. It stores the alignment path but not the sequences.

In [ ]:
from skbio.alignment import AlignPath, PairAlignPath

In [ ]:
path = PairAlignPath.from_tabular(align)
path

The underlying data structure of an alignment path is two arrays: `lengths` and `states`. The `lengths` array represents the segment length for each segment with consistent gap status. The sum of the `lengths` array is equal to the number of the positions in the alignment, in our case 588.

In [ ]:
path.lengths

In [ ]:
path.lengths.sum()

The `states` array represents the gap status of each individual segment in the alignment. For a pairwise alignment `0` means (mis)match, `1` means insertion, and `2` means deletion.

In [ ]:
path.states

A pairwise alignment path can be converted into a [CIGAR string](https://en.wikipedia.org/wiki/Sequence_alignment#CIGAR_Format).

In [ ]:
path.to_cigar()

If greater resolution is desired between match and mismatch (`=` and `X` vs `M` in CIGAR strings), you may pass the sequences to `to_cigar`.

In [ ]:
path.to_cigar(seqs=[align[0], align[1]])

### Substitution matrix

For protein sequence alignment, a [**substitution matrix**](https://en.wikipedia.org/wiki/Substitution_matrix) is usually used to define the scores of changes between amino acids. scikit-bio's [`SubstitutionMatrix`](https://scikit.bio/docs/latest/generated/skbio.sequence.SubstitutionMatrix.html) class provides this functionality.

In [ ]:
from skbio import SubstitutionMatrix

Load a pre-defined substitution matrix, for example [BLOSUM62](https://en.wikipedia.org/wiki/BLOSUM). 

In [ ]:
sm = SubstitutionMatrix.by_name('BLOSUM62')

The substitution matrix can be fed into the alignment algorithm.

In [ ]:
sm_dict = substitution_matrix=sm.to_dict()

In [ ]:
align, score, pos = local_pairwise_align_ssw(
    human.translate(), mouse.translate(), substitution_matrix=sm_dict)
align

## Multiple Alignment

It is also possible to use scikit-bio to perform [multiple sequence alignment](https://en.wikipedia.org/wiki/Multiple_sequence_alignment). The following code demonstrates the procedures of [**progressive alignment**](https://en.wikipedia.org/wiki/Multiple_sequence_alignment#Progressive_alignment_construction) -- a classical strategy for aligning multiple sequences.

Here, we will first translate the sequences into protein, and align the protein sequences from the seven organisms. We also create a dictionary of the sequences where each key is the name of the organism, and each value is the `Protein` sequence of that organism. This step is necessary for the progressive alignment function we've written. As you see below, translation of all the sequences is possible in a single line of code using list comprehension.

In [ ]:
prot_seqs = [seq.translate() for seq in seqs]

In [ ]:
names = ('human', 'mouse', 'rat', 'chicken', 'pig', 'chimp', 'monkey')

In [ ]:
prot_seqs_dict = {name: sequence for name, sequence in zip(names, prot_seqs)}

In [ ]:
prot_seqs_dict['pig']

### Distance matrix

Now with all of our sequences translated we can build a [**distance matrix**](https://en.wikipedia.org/wiki/Distance_matrix) among them. scikit-bio's [`DistanceMatrix`](https://scikit.bio/docs/latest/generated/skbio.stats.distance.DistanceMatrix.html) class provides a convenient interface for working with distance matrices. Among its various functionality, we will use the [`from_iterable`](https://scikit.bio/docs/latest/generated/skbio.stats.distance.DistanceMatrix.from_iterable.html) method to automatically create a distance matrix for all pairs of items in a Python iterable, in this case the list of sequences.

In [ ]:
from skbio import DistanceMatrix

We will define a function that calculates the distance between a pair of items. Here, we will perform pairwise sequence alignment, then calculate the [Hamming distance](https://en.wikipedia.org/wiki/Hamming_distance) between the aligned sequences.

In [ ]:
from skbio.sequence.distance import hamming

In [ ]:
def align_dist(seq1, seq2):
    aln = local_pairwise_align_ssw(seq1, seq2, substitution_matrix=sm_dict)[0]
    return hamming(aln[0], aln[1])

Build a distance using just one line of code:

In [ ]:
dm = DistanceMatrix.from_iterable(prot_seqs, align_dist, key='id', validate=False)

By typing `dm` you will see a heatmap representing the distance matrix.

In [ ]:
dm

The underlying data of the distance matrix is a 2D array.

In [ ]:
dm.data

Pairwise distances can be accessed using their IDs.

In [ ]:
dm['human', 'mouse']

### Guide tree

Once we've created our distance matrix, we can generate a tree based on this matrix using any [distance-based methods](https://en.wikipedia.org/wiki/Computational_phylogenetics#Distance-matrix_methods). This tree will serve as the _guide tree_ for the progressive alignment algorithm.

We will use SciPy's [`average`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.average.html) function, which implements the [**UPGMA**](https://en.wikipedia.org/wiki/UPGMA) method (a hierarchical clustering method that is also widely used in phylogenetics). The return value is a _linkage matrix_.

In [ ]:
from scipy.cluster.hierarchy import average

In [ ]:
guide_lm = average(dm.condensed_form())

scikit-bio's [`TreeNode`](https://scikit.bio/docs/latest/generated/skbio.tree.TreeNode.html) class is a versatible interface for working with tree structures, with various enhancements for _phylogenetic trees_. It can convert a linkage matrix into a tree.

In [ ]:
from skbio import TreeNode

In [ ]:
guide_tree = TreeNode.from_linkage_matrix(guide_lm, dm.ids)

Print an ASCII art of the tree. Does it make sense?

In [ ]:
print(guide_tree.ascii_art())

### Progressive alignment

Here we define our progressive alignment function. In this algorithm, proximal sequences and groups of sequences are sequentially merged via pairwise alignment, following the order defined by the guide tree. The slower but more versatible `global_pairwise_align_protein` function will be used.

In [ ]:
from skbio.alignment import global_pairwise_align_protein

In [ ]:
def progressive_msa(query_seqs, guide_tree):
    c1, c2 = guide_tree.children

    if c1.is_tip():
        c1_aln = query_seqs[c1.name]
    else:
        c1_aln = progressive_msa(query_seqs, c1)

    if c2.is_tip():
        c2_aln = query_seqs[c2.name]
    else:
        c2_aln = progressive_msa(query_seqs, c2)

    return global_pairwise_align_protein(
        c1_aln, c2_aln, substitution_matrix=sm_dict)[0]

Here we perform progressive alignment. By default, the alignment programs in scikit-bio will return `TabularMSA` objects. As mentioned previously, these are intuitive data structures and have several built-in methods which are useful.

In [ ]:
msa = progressive_msa(prot_seqs_dict, guide_tree)

In [ ]:
msa

We can also convert the `TabularMSA` object into an `AlignPath` object like we did earlier. `AlignPath` objects are exactly like `PairAlignPath` objects except that they don't have a few methods which are specific to pairwise alignments (e.g., CIGAR). The underlying data structure is the same, and can be quite useful.

In [ ]:
msa_path = AlignPath.from_tabular(msa)

In [ ]:
msa_path.lengths

In [ ]:
msa_path.states

## Alignment exploration

We can next get some information about this alignment. For example, we can easily see that there are 7 sequences in this alignment, and that it's 276 characters long. By definition, there is no variance in sequence length in a multiple sequence alignment. For this reason, it's common to think of an alignment as a matrix or table, where rows represent sequences and columns represent positions in the sequences.

In [ ]:
msa.shape

In [ ]:
msa.index

Here we can see what the consensus sequence among the alignment is by calling the `consensus` method.

In [ ]:
msa.consensus()

We can also explore some more interesting features of this alignment. For example, we can compute [**conservation**](https://en.wikipedia.org/wiki/Conserved_sequence) for each position in the alignment using the [`conservation`](https://scikit.bio/docs/latest/generated/skbio.alignment.TabularMSA.conservation.html) method. It calculates positional conservation using the inverse Shannon uncertainty metric.

In [ ]:
conserv = msa.conservation(gap_mode='include')

We get an array of positional conservation values (here we'll just print the first ten). A low conservation value means that there is a lot of variation in the sequences at the corresponding position in the alignment, while a high conservation value means that the corresponding position in the alignment is highly conserved. A conservation of 1.0 means that a position is perfectly conserved. A conservation of 0.0 means that every character in the alphabet is present in exactly equal frequency at that position in the alignment (this is uncommon in practice).

In [ ]:
conserv[100:110]

In our alignment, there are 28 positions that are perfectly conserved across all sequences.

In [ ]:
(conserv == 1.0).sum()

Let's figure out what amino acids are at these perfectly conserved positions:

In [ ]:
res = []
for i, value in enumerate(conserv):
    if value == 1.0:
        res.append((i, str(msa[0][i])))

In [ ]:
print(res)

Finally we can visualize the conservation along the full length of the alignment to get an idea of where the more and less conserved positions are found. In general, positions that are highly conserved are thought to have specific structural or functional roles in the protein, while positions that are less conserved might have less specific roles (e.g., providing a filler between two functional domains in the linear protein backbone).

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(12, 3))
plt.plot(range(len(conserv)), conserv)
plt.xlabel('Position')
plt.ylabel('Conservation')
plt.title('Positional Conservation Plot');

## Phylogenetics

With a multiple sequence alignment, one can reconstruct a [**phylogenetic tree**](https://en.wikipedia.org/wiki/Phylogenetic_tree) to model the evolutionary relationships among the organisms. A variety of computational methods exist for this task. Here, we will demonstrate one of the simplest yet scalable methods.

First, let's also build a distance matrix based on the Hamming distances among alignment sequences. The difference of this distance matrix from the previous one is that it is based on the multiple sequence alignment , rather than separate pairwise alignments. Therefore it is presumably more accurate.

In [ ]:
dm_msa = DistanceMatrix.from_iterable(msa, hamming, key='id', validate=False)
dm_msa

Then we will call [**neighbor joining**](https://en.wikipedia.org/wiki/Neighbor_joining), a classical distance-based phylogenetic reconstruction method. It is implemented in scikit-bio's [`nj`](https://scikit.bio/docs/latest/generated/skbio.tree.nj.html) function.

In [ ]:
from skbio.tree import nj

In [ ]:
tree = nj(dm)

Unlike UPGMA, the output of neighbor-joining is an unrooted tree. We will perform mid-point rooting on it.

In [ ]:
tree = tree.root_at_midpoint()

Explore the tree. One can assess whether / to what extent the tree topology aligns with our knowledge of vertebrate evolution, and make relevant interpretations.

In [ ]:
print(tree.ascii_art())

## Summary

This first section of the scikit-bio tutorial introduces the installation of scikit-bio, the setup of the practice environment, and demonstrates the basic usage of scikit-bio via a series of bioinformatics analyses: sequence, alignment, and phylogeny. The content serves as the basis for more sophisticated omic data analyses, as will be demonstrated in subsequent sections.